In [1]:
import json

# COCO 데이터셋에서 이미지와 annotation 정보를 읽어옴
with open('/opt/ml/dataset/train.json', 'r') as f:
    coco_data = json.load(f)

# 유니크한 클래스 개수가 6개 이상인 이미지와 bbox 개수가 30개 이상인 이미지를 담을 리스트
unique_class_images = []
bbox_30_images = []

for img in coco_data['images']:
    bbox_num = []
    category_num = []
    img_id = img['id']
    # ann_ids = coco_data['imgToAnns'].get(str(img_id), [])
    anns = [ann for ann in coco_data['annotations'] if ann['image_id'] == img_id]
    ann_ids = anns
    # print(ann_ids)
    # break
    bbox_count = len(ann_ids)
    if bbox_count >= 30:
        bbox_30_images.append(img['file_name'])
        
    else:
        category_count = len(set(ann['category_id'] for ann in ann_ids))
        if category_count >= 6:
            unique_class_images.append(img['file_name'])

outlier_images = list(set(bbox_30_images+unique_class_images))
# 결과 출력
print("bbox 개수가 30개 이상인 이미지 이름: ", bbox_30_images)
print("bbox_30_images 이미지 개수:", len(bbox_30_images))
print("유니크 클래스 개수가 6개 이상인 이미지 이름: ", unique_class_images)
print("unique_class_images 이미지 개수:", len(unique_class_images))
print("bbox 개수가 30개 이상이거나 유니크 클래스 개수가 6개 이상인 이미지:", outlier_images)
print("outlier_images 이미지 개수:", len(outlier_images))


bbox 개수가 30개 이상인 이미지 이름:  ['train/0168.jpg', 'train/0272.jpg', 'train/0321.jpg', 'train/0352.jpg', 'train/0384.jpg', 'train/0394.jpg', 'train/0624.jpg', 'train/0655.jpg', 'train/0694.jpg', 'train/0767.jpg', 'train/0833.jpg', 'train/0840.jpg', 'train/0946.jpg', 'train/0956.jpg', 'train/1002.jpg', 'train/1020.jpg', 'train/1058.jpg', 'train/1177.jpg', 'train/1669.jpg', 'train/1671.jpg', 'train/1678.jpg', 'train/1797.jpg', 'train/1872.jpg', 'train/1918.jpg', 'train/1955.jpg', 'train/1997.jpg', 'train/2182.jpg', 'train/2189.jpg', 'train/2387.jpg', 'train/2416.jpg', 'train/2521.jpg', 'train/2572.jpg', 'train/2678.jpg', 'train/2794.jpg', 'train/2956.jpg', 'train/2981.jpg', 'train/3026.jpg', 'train/3046.jpg', 'train/3049.jpg', 'train/3135.jpg', 'train/3148.jpg', 'train/3403.jpg', 'train/3430.jpg', 'train/3450.jpg', 'train/3601.jpg', 'train/3716.jpg', 'train/3796.jpg', 'train/3825.jpg', 'train/4047.jpg', 'train/4119.jpg', 'train/4197.jpg', 'train/4225.jpg', 'train/4261.jpg', 'train/4492.jpg', '

## 모든 이미지의 경로, 총 bbox, 유니크 클래스 개수, 클래스 명 CSV 파일

In [2]:
import json
import csv

# COCO 데이터셋 JSON 파일 경로
coco_json_file = '/opt/ml/dataset/train.json'

# COCO 데이터셋 JSON 파일 읽기
with open(coco_json_file, "r") as f:
    coco_data = json.load(f)

# 이미지 정보와 객체 정보를 읽어서 딕셔너리에 저장
images = {}
for img in coco_data["images"]:
    img_id = img["id"]
    images[img_id] = {
        "filename": img["file_name"],
        "bbox_count": 0,
        "class_count": 0,
        "classes": set()
    }
for obj in coco_data["annotations"]:
    img_id = obj["image_id"]
    images[img_id]["bbox_count"] += 1
    class_id = obj["category_id"]
    class_name = coco_data["categories"][class_id]["name"]
    images[img_id]["classes"].add(class_name)

# 유니크한 클래스 개수와 클래스 이름을 계산
for img_data in images.values():
    img_data["class_count"] = len(img_data["classes"])
    img_data["classes"] = ", ".join(sorted(img_data["classes"]))

# CSV 파일 생성
with open("image_stats.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Filename", "Bounding Box Count", "Class Count", "Classes"])
    for img_id, img_data in images.items():
        writer.writerow([img_data["filename"], img_data["bbox_count"], img_data["class_count"], img_data["classes"]])

print("Done!")


Done!


## outlier 이미지의 경로, 총 bbox, 유니크 클래스 개수, 클래스 명 CSV 파일

In [3]:
import json
import csv

# COCO 데이터셋 JSON 파일 경로
coco_json_file = '/opt/ml/dataset/train.json'

# 추출할 이미지 파일 리스트
outlier_images.sort()

# COCO 데이터셋 JSON 파일 읽기
with open(coco_json_file, "r") as f:
    coco_data = json.load(f)

# 이미지 정보와 객체 정보를 읽어서 딕셔너리에 저장
images = {}
for img in coco_data["images"]:
    if img["file_name"] in outlier_images:
        img_id = img["id"]
        images[img_id] = {
            "filename": img["file_name"],
            "bbox_count": 0,
            "class_count": 0,
            "classes": set()
        }
for obj in coco_data["annotations"]:
    img_id = obj["image_id"]
    if img_id in images:
        images[img_id]["bbox_count"] += 1
        class_id = obj["category_id"]
        class_name = coco_data["categories"][class_id]["name"]
        images[img_id]["classes"].add(class_name)

# 유니크한 클래스 개수와 클래스 이름을 계산
for img_data in images.values():
    img_data["class_count"] = len(img_data["classes"])
    img_data["classes"] = ", ".join(sorted(img_data["classes"]))

# CSV 파일 생성
with open("image_stats_selected.csv", "w", newline="") as f:
    writer = csv.writer(f)
    writer.writerow(["Filename", "Bounding Box Count", "Class Count", "Classes"])
    for img_id, img_data in images.items():
        writer.writerow([img_data["filename"], img_data["bbox_count"], img_data["class_count"], img_data["classes"]])

print("Done!")

Done!
